In [1]:
import os
# Keras에서 발생하는 경고 메시지 제거
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
tf.random.set_seed(22)
np.random.seed(22)
assert tf.__version__.startswith('2.')

batch_size =128
total_words = 10000
max_review_len = 80
embedding_len = 100

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words= total_words)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(10000).batch(batch_size, drop_remainder= True)

test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(batch_size, drop_remainder= True)

print('x_train_shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test_shape:', x_test.shape)

sample = next(iter(test_data))
print(sample[0].shape)

x_train_shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test_shape: (25000, 80)
(128, 80)


In [4]:
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()

        self.state0 = [tf.zeros([batch_size, units])]
        self.state1 = [tf.zeros([batch_size, units])]
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length= max_review_len)

        self.RNNCell0 = tf.keras.layers.SimpleRNNCell(units, dropout= 0.2)
        self.RNNCell1 = tf.keras.layers.SimpleRNNCell(units, dropout= 0.2)
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training= None):
        x = inputs
        x = self.embedding(x)

        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis= 1):
            out0, state0 = self.RNNCell0(word, state0, training)
            out1, state1 = self.RNNCell1(out0, state1, training)

        x = self.outlayer(out1)
        prob = tf.sigmoid(x)
        return prob

In [5]:
import time

In [6]:
units = 64
epochs = 4
t0 = time.time()

model = RNN_Build(units)
model.compile(
    optimizer= tf.keras.optimizers.Adam(0.001),
    loss= tf.losses.BinaryCrossentropy(),
    metrics= ['accuracy'],
    experimental_run_tf_function= False
)

model.fit(train_data, epochs= epochs, validation_data= test_data, validation_freq= 2)

Epoch 1/4
195/195 [==============================] - 18s 93ms/step - loss: 0.5203 - accuracy: 0.6226
Epoch 2/4
195/195 [==============================] - 13s 69ms/step - loss: 0.3192 - accuracy: 0.8699 - val_loss: 0.4004 - val_accuracy: 0.8190
Epoch 3/4
195/195 [==============================] - 9s 48ms/step - loss: 0.2112 - accuracy: 0.9209
Epoch 4/4
195/195 [==============================] - 12s 60ms/step - loss: 0.1113 - accuracy: 0.9594 - val_loss: 0.6613 - val_accuracy: 0.8212


In [7]:
print("훈련 데이터셋 평가")
(loss, accuracy) = model.evaluate(train_data, verbose= 0)

print("loss= {:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

print("테스트 데이터셋 평가")
(loss, accuracy) = model.evaluate(test_data, verbose= 0)

print("loss= {:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가
loss= 0.0518, accuracy: 98.0809%
시간: 55.273107290267944
테스트 데이터셋 평가
loss= 0.6613, accuracy: 82.1154%
시간: 58.11088967323303


In [14]:
class RNN_Build(tf.keras.Model):
    def __init__(self, units):
        super(RNN_Build, self).__init__()
        self.embedding = tf.keras.layers.Embedding(total_words, embedding_len, input_length= max_review_len)

        self.rnn = tf.keras.Sequential([
            tf.keras.layers.SimpleRNN(units, dropout= 0.5, return_sequences= True),
            tf.keras.layers.SimpleRNN(units, dropout= 0.5)
        ])
        self.outlayer = tf.keras.layers.Dense(1)

    def call(self, inputs, training= None):
        x = inputs
        x = self.embedding(x)
        x = self.rnn(x)
        x = self.outlayer(x)
        prob = tf.sigmoid(x)

        return prob

In [15]:
units = 64
epochs = 4
t0 = time.time()

model = RNN_Build(units)
model.compile(
    optimizer= tf.keras.optimizers.Adam(0.001),
    loss= tf.losses.BinaryCrossentropy(),
    metrics= ['accuracy'],
    experimental_run_tf_function= False
)

model.fit(train_data, epochs= epochs, validation_data= test_data, validation_freq= 2)

Epoch 1/4
195/195 [==============================] - 17s 88ms/step - loss: 0.6120 - accuracy: 0.5522
Epoch 2/4
195/195 [==============================] - 22s 110ms/step - loss: 0.3978 - accuracy: 0.8123 - val_loss: 0.4403 - val_accuracy: 0.8212
Epoch 3/4
195/195 [==============================] - 16s 83ms/step - loss: 0.3247 - accuracy: 0.8678
Epoch 4/4
195/195 [==============================] - 22s 113ms/step - loss: 0.2728 - accuracy: 0.8975 - val_loss: 0.4583 - val_accuracy: 0.8253


In [17]:
print("훈련 데이터셋 평가")
(loss, accuracy) = model.evaluate(train_data, verbose= 0)

print("loss= {:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

print("테스트 데이터셋 평가")
(loss, accuracy) = model.evaluate(test_data, verbose= 0)

print("loss= {:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))
t1 = time.time()
print('시간:', t1-t0)

훈련 데이터셋 평가
loss= 0.1981, accuracy: 92.3357%
시간: 104.54955339431763
테스트 데이터셋 평가
loss= 0.4583, accuracy: 82.5321%
시간: 109.90021848678589
